In [1]:
import cv2
import numpy as np
import mysql.connector
import json
import face_recognition
from flask import Flask, request, jsonify, render_template, redirect, url_for

app = Flask(__name__)

# âœ… Database Configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "police_case_management"
}

def connect_db():
    """Ensures database connection is always valid."""
    try:
        conn = mysql.connector.connect(**db_config)
        return conn
    except mysql.connector.Error as err:
        print(f"Database Error: {err}")
        return None

def capture_face():
    """Attempts to capture an image frame using OpenCV."""
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # âœ… Fix for MSMF errors

    if not video_capture.isOpened():
        return None, "Error: Unable to open webcam"

    for _ in range(10):  # âœ… Allow time for camera initialization
        ret, frame = video_capture.read()
        if ret:
            break
        cv2.waitKey(100)

    video_capture.release()

    if not ret:
        return None, "Error: Failed to capture image from webcam"

    return frame, None

@app.route('/')
def base():
    """Serve the face login page."""
    return render_template('login_face.html')

@app.route('/register_face', methods=['POST'])
def register_face():
    """Stores user's face encoding in the database."""
    username = request.form['username']
    role = request.form['role']

    frame, error_message = capture_face()
    if error_message:
        return jsonify({"error": error_message}), 500

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    if not face_encodings:
        return jsonify({"error": "No face detected"}), 400

    encoding_str = json.dumps(face_encodings[0].tolist())  # âœ… Convert to JSON string for MySQL storage

    conn = connect_db()
    if conn is None:
        return jsonify({"error": "Database connection failed"}), 500

    cursor = conn.cursor()
    query = "INSERT INTO users (username, role, face_encoding) VALUES (%s, %s, %s)"
    cursor.execute(query, (username, role, encoding_str))
    conn.commit()
    conn.close()

    return jsonify({"success": "User registered successfully"}), 200

@app.route('/login_face', methods=['POST'])
def login_face():
    """Validates user by comparing stored face encoding."""
    frame, error_message = capture_face()
    if error_message:
        return jsonify({"error": error_message}), 500

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    if not face_encodings:
        return jsonify({"error": "No face detected"}), 400

    conn = connect_db()
    if conn is None:
        return jsonify({"error": "Database connection failed"}), 500

    cursor = conn.cursor()
    cursor.execute("SELECT id, username, role, face_encoding FROM users")
    users = cursor.fetchall()
    conn.close()

    for user in users:
        try:
            user_encoding = np.array(json.loads(user[3]))  # âœ… Convert JSON string back to numpy array
            match = face_recognition.compare_faces([user_encoding], face_encodings[0])[0]

            if match:
                return jsonify({
                    "success": True,
                    "username": user[1],
                    "role": user[2],
                    "redirect": url_for('dashboard', view='cards') if user[2] == "admin" else url_for('dashboard', view='officer')
                }), 200
        except ValueError:
            continue  # Skip invalid stored encodings

    return jsonify({"error": "Face not recognized"}), 403

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
